In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import ast
from datetime import datetime

import sys
print(sys.version)

3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 08:57:19) [GCC 11.3.0]


In [3]:
# Establish snowflake connection
conn = snowflake.connector.connect(
    account = "carelon-edaprod1.privatelink",
    user = "AN587958AD",
    password = "^qf5m3nD47h",
    warehouse = "DL_AIFS_STAR_USER_WH_L",
    database = "NON_CRTFD_AIFS",
    schema = "DL_TS_STAR"
)

# Create a cursor object
cursor = conn.cursor()

In [3]:
#read input dataset v2 table
query= """select * from NON_CRTFD_AIFS.DL_TS_STAR_NOGBD.ihan_hcc136to138_dataset_v2"""
df = pd.read_sql_query(query, conn)
df.head()

/home/jovyan/tmp/ipykernel_8481/692520890.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


MCID                                               DOS1  \
0  220720023  ['2017-03-13', '2017-04-17', '2017-04-25', '20...   
1  629357561         ['2018-11-10', '2018-12-17', '2019-01-09']   
2  372093659  ['2017-01-03', '2017-01-04', '2017-01-05', '20...   
3   54758643  ['2019-02-19', '2019-02-26', '2019-03-05', '20...   
4  220554262  ['2017-05-10', '2017-11-29', '2017-12-08', '20...   

                                            DATLIST1  \
0  [[14104], [13642, 42211], [13642, 14172, 42211...   
1  [[2758, 2907, 19502, 41554, 41584, 41665], [29...   
2  [[1123, 41928], [1123, 41928], [1123, 41928], ...   
3  [[3656, 3732, 8340, 9815, 19359, 19588, 19593]...   
4  [[8340, 10508], [41438], [41538, 42187], [8340...   

                                                DOS2  \
0  ['2017-03-13', '2017-06-13', '2017-07-03', '20...   
1         ['2018-11-10', '2018-12-17', '2019-01-09']   
2  ['2017-01-03', '2017-01-04', '2017-01-05', '20...   
3  ['2019-02-19', '2019-02-26', '2019-03-05', '20...   
4  ['2017-11-29', '2017-12-08', '2017-12-19', '20...   

                                            DATLIST2            DOS3  \
0  [[12281], [10040, 12281], [3930, 7729, 8403, 8...              []   
1  [[7729, 8024, 8193, 9121, 10042, 12111], [1003...              []   
2  [[7550, 7566, 7574, 7579, 7580], [7550, 7566, ...  ['2019-08-07']   
3  [[3930, 7729, 7731, 7853, 8115, 8145, 8354, 84...              []   
4  [[12262], [7539, 12166], [10040], [3930, 7726,...              []   

   DATLIST3                                               DOS4  \
0        []  ['2017-05-22', '2017-05-23', '2017-06-13', '20...   
1        []                       ['2018-11-10', '2019-01-10']   
2  [[9602]]  ['2017-05-30', '2017-07-28', '2017-08-04', '20...   
3        []  ['2019-03-08', '2019-03-15', '2019-05-22', '20...   
4        []  ['2017-06-09', '2017-09-22', '2017-10-18', '20...   

                                            DATLIST4  NUM_AGE  NUM_GENDERM  \
0  [[590, 1298], [226], [1084], [590, 1298], [471...       72            0   
1                          [[465], [465, 859, 1094]]       83            0   
2  [[414, 436, 447, 623], [1082], [769], [769], [...       72            1   
3  [[929], [525, 530, 547, 622], [525, 530, 547, ...       83            0   
4  [[578], [578], [177], [191], [578], [578], [41...       82            0   

   NUM_TARGET_HCC136_ONLY_V4  NUM_TARGET_HCC137_ONLY_V4  \
0                          0                          0   
1                          0                          0   
2                          0                          0   
3                          0                          0   
4                          0                          0   

   NUM_TARGET_HCC138_ONLY_V4       LAB_DOS    LABPAIRLIST  
0                          0            []             []  
1                          0            []             []  
2                          0  [2019-08-07]  [{9602: 0.0}]  
3                          0            []             []  
4                          1            []             []

In [4]:
df.shape

(824099, 16)

In [5]:
df_sample=df.sample(n=50000,random_state=1)

In [6]:
df_sample.duplicated().any()

False

In [7]:
#preparing the data
def process_medical_data(row):
    """
    Process medical data for a single row, handling multiple types of medical records
    including lab results with values
    
    Parameters:
    row: pandas Series containing patient data
    
    Returns:
    list of dictionaries containing processed records
    """
    # Common fields for all records
    mcid = row['MCID']
    age = row['NUM_AGE']
    gender = row['NUM_GENDERM']
    
    results = []
    
    # Process diag data
    if 'DOS1' in row and 'DATLIST1' in row:
        dates = ast.literal_eval(row['DOS1'])
        codes_list = ast.literal_eval(row['DATLIST1'])
        for date, codes in zip(dates, codes_list):
            for code in codes:
                results.append({
                    'mcid': mcid,
                    'dos': date,
                    'codeIndex': int(code),
                    'age': age,
                    'gender': gender,
                    'type': 'diag',
                    'value': None
                })
    
    # Process procedure data
    if 'DOS2' in row and 'DATLIST2' in row:
        dates = ast.literal_eval(row['DOS2'])
        codes_list = ast.literal_eval(row['DATLIST2'])
        for date, codes in zip(dates, codes_list):
            for code in codes:
                results.append({
                    'mcid': mcid,
                    'dos': date,
                    'codeIndex': int(code),
                    'age': age,
                    'gender': gender,
                    'type': 'proc',
                    'value': None
                })
    
    # Process lab data with values
    if 'DOS3' in row and 'LABPAIRLIST' in row:
        dates = ast.literal_eval(row['DOS3'])
        lab_pairs = ast.literal_eval(row['LABPAIRLIST'])
        for date, lab_dict in zip(dates, lab_pairs):
            for lab_code, lab_result in lab_dict.items():
                results.append({
                    'mcid': mcid,
                    'dos': date,
                    'codeIndex': int(lab_code),
                    'value': lab_result,
                    'age': age,
                    'gender': gender,
                    'type': 'lab'
                })
    
    # Process rx data
    if 'DOS4' in row and 'DATLIST4' in row:
        dates = ast.literal_eval(row['DOS4'])
        codes_list = ast.literal_eval(row['DATLIST4'])
        for date, codes in zip(dates, codes_list):
            for code in codes:
                results.append({
                    'mcid': mcid,
                    'dos': date,
                    'codeIndex': int(code),
                    'age': age,
                    'gender': gender,
                    'type': 'gpi',
                    'value': None
                })
    
    return results

# Process all rows
result_list = []
for _, row in df_sample.iterrows():
    result_list.extend(process_medical_data(row))

# Create the final combined dataframe
result_df = pd.DataFrame(result_list)

In [8]:
result_df['type'].value_counts()

type
diag    7593330
proc    6003877
gpi     3192637
lab     2972668
Name: count, dtype: int64

## reading dictionary files 

In [9]:
#read diag dict file
dict_diag=pd.read_csv('/home/jovyan/data/ihan_paper/data_v2/codeDict/codeDict_diag_claimEMR_hcc138.csv')
dict_diag.head()

idx cd_value cd_type DIAG_CD RPTG_DIAG_CD SHRT_CD_DEFN_TXT ICD_FULL_DESC  \
0    1       00      dx     NaN          NaN              NaN           NaN   
1    2       01      dx     NaN          NaN              NaN           NaN   
2    3    02401      dx     NaN          NaN              NaN           NaN   
3    4    02831      dx     NaN          NaN              NaN           NaN   
4    5    02834      dx     NaN          NaN              NaN           NaN   

  ICD_CTGRY_1_TXT  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN

In [10]:
#read proc dict file
dict_proc=pd.read_csv('/home/jovyan/data/ihan_paper/data_v2/codeDict/codeDict_proc_claimEMR_hcc138.csv')
dict_proc.head()

idx cd_value cd_type              shrt_cd_defn_txt  \
0  42685      000   hcpcs                           NaN   
1  42686    00000   hcpcs                           NaN   
2  42687    0001F   hcpcs       HEART FAILURE COMPOSITE   
3  42688    0001M   hcpcs    NFCT DS CHRNC HCV 6 ASSAYS   
4  42689    0001U   hcpcs  RBC DNA HEA 35 AG 11 BLD GRP   

                                 medm_cd_defn_txt  \
0                                             NaN   
1                                             NaN   
2                            HRT FAILURE ASSESSED   
3   NFCT DS CHRNC HCV 6 BIOCHEM ASSAY SRM ALG LVR   
4  RBC DNA HEA 35 AG 11 BLD GRP WHL BLD CMN ALLEL   

                                        proc_cd_desc  
0                                                NaN  
1                                                NaN  
2  Heart failure assessed (includes assessment of...  
3  Infectious disease, chronic hepatitis C virus ...  
4  Red blood cell antigen typing, DNA, human eryt...

In [11]:
#read lab dict file
dict_lab=pd.read_csv('/home/jovyan/data/ihan_paper/data_v2/codeDict/codeDict_lab_claimEMR_hcc138.csv')
dict_lab.head()

idx    cd_value cd_type      cd abn_cd  \
0  71917       0_UNK   loinc       0    UNK   
1  71918    1004-1_A   loinc  1004-1      A   
2  71919  1004-1_UNK   loinc  1004-1    UNK   
3  71920    1006-6_A   loinc  1006-6      A   
4  71921  1006-6_UNK   loinc  1006-6    UNK   

                                    LOINC_CMPNT_DESC CLSFCTN_NM  \
0                                                NaN        NaN   
1  Direct antiglobulin test.complement specific r...      BLDBK   
2  Direct antiglobulin test.complement specific r...      BLDBK   
3      Direct antiglobulin test.IgG specific reagent      BLDBK   
4      Direct antiglobulin test.IgG specific reagent      BLDBK   

             LOINC_SHRT_NM  
0                      NaN  
1  DAT Comp-Sp Reag RBC Ql  
2  DAT Comp-Sp Reag RBC Ql  
3  DAT IgG-Sp Reag RBC-Imp  
4  DAT IgG-Sp Reag RBC-Imp

In [12]:
#read lab pair file
dict_lab_pair=pd.read_csv('/home/jovyan/data/ihan_paper/data_v2/codeDict/codeIdx_labPair_claim.csv')
dict_lab_pair.head()

idx loinc_cd abnrml_cd  result_mean  result_stddev
0  72583  13502-0         A          NaN            NaN
1  77209  33379-9       UNK          NaN            NaN
2  78861   4677-1         L      51.3625      15.124053
3  81204   6174-7         N       0.1000       0.000000
4  75763  26600-7         H       3.6000            NaN

In [13]:
#read rx dict file
dict_rx=pd.read_csv('/home/jovyan/data/ihan_paper/data_v2/codeDict/codeDict_rx_claimEMR_hcc138.csv')
dict_rx.head()

idx  cd_value cd_type                CD_VAL_NM         CD_VAL_SHRT_DESC  \
0  84181  00000000     gpi                      NaN                      NaN   
1  84182  01100010     gpi             Penicillin G             Penicillin G   
2  84183  01100020     gpi  Penicillin G Benzathine  Penicillin G Benzathine   
3  84184  01100030     gpi    Penicillin G Procaine    Penicillin G Procaine   
4  84185  01100040     gpi             Penicillin V             Penicillin V   

          CD_VAL_LONG_DESC  
0                      NaN  
1             Penicillin G  
2  Penicillin G Benzathine  
3    Penicillin G Procaine  
4             Penicillin V

In [14]:
#merging with code dictionaries
def idx2code(df, codedict, cols,idx_offset):
    """
    Merge the code index with the corresponding code and code_desc in codedict when the indecies are in one contigouous blocks
    codedict: with columns in cols 
    idx_offset: offset between index in df and index in codedict
    """
    df2 = codedict[cols]
    df2.columns = ['codeIndex','code','cd_desc']
    df['codeIndex'] = df['codeIndex'] + idx_offset
    df = pd.merge(df, df2, on=['codeIndex'], how = 'left')
    df['abnormal_cd']=None
    return df

In [15]:
#merging for diag code
result_df_diag=result_df[result_df['type']=='diag']
merged_df_diag=idx2code(result_df_diag,dict_diag,cols=['idx','DIAG_CD','ICD_FULL_DESC'],idx_offset=0)
merged_df_diag.head()

/home/jovyan/tmp/ipykernel_8481/424982093.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['codeIndex'] = df['codeIndex'] + idx_offset


mcid         dos  codeIndex  age  gender  type  value   code  \
0  334213630  2017-10-16       2904   77       1  diag    NaN   E034   
1  334213630  2017-10-16       3648   77       1  diag    NaN  E7800   
2  334213630  2017-10-16      19659   77       1  diag    NaN  R7303   
3  334213630  2017-10-16      19863   77       1  diag    NaN  R9720   
4  334213630  2017-10-16      41665   77       1  diag    NaN    Z23   

                                   codeDesc abnormal_cd  
0             Atrophy of thyroid (acquired)        None  
1    Pure hypercholesterolemia, unspecified        None  
2                               Prediabetes        None  
3  Elevated prostate specific antigen [PSA]        None  
4                Encounter for immunization        None

In [16]:
#merge with proc dict file
result_df_proc=result_df[result_df['type']=='proc']
merged_df_proc=idx2code(result_df_proc,dict_proc,cols=['idx','cd_value','proc_cd_desc'],idx_offset=42684)
merged_df_proc.head()

/home/jovyan/tmp/ipykernel_8481/424982093.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['codeIndex'] = df['codeIndex'] + idx_offset


mcid         dos  codeIndex  age  gender  type  value   code  \
0  334213630  2017-10-16      51795   77       1  proc    NaN  90670   
1  334213630  2017-10-16      51805   77       1  proc    NaN  90686   
2  334213630  2017-10-16      52725   77       1  proc    NaN  99214   
3  334213630  2018-08-06      49838   77       1  proc    NaN  71046   
4  334213630  2018-08-06      52317   77       1  proc    NaN  94640   

                                            codeDesc abnormal_cd  
0  Pneumococcal conjugate vaccine, 13 valent (PCV...        None  
1  Influenza virus vaccine, quadrivalent (IIV4), ...        None  
2  Office or other outpatient visit for the evalu...        None  
3             Radiologic examination, chest; 2 views        None  
4  Pressurized or nonpressurized inhalation treat...        None

In [17]:
#merge with RX dict data 
result_df_gpi=result_df[result_df['type']=='gpi']
merged_df_gpi=idx2code(result_df_gpi,dict_rx,cols=['idx','cd_value','CD_VAL_SHRT_DESC'],idx_offset=84180)
merged_df_gpi.head()

/home/jovyan/tmp/ipykernel_8481/424982093.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['codeIndex'] = df['codeIndex'] + idx_offset


mcid         dos  codeIndex  age  gender type  value      code  \
0  334213630  2018-05-02      84645   77       1  gpi    NaN  28100010   
1  334213630  2018-07-08      84645   77       1  gpi    NaN  28100010   
2  334213630  2018-08-07      84223   77       1  gpi    NaN  05000034   
3  334213630  2018-10-05      84645   77       1  gpi    NaN  28100010   
4  334213630  2018-12-07      84554   77       1  gpi    NaN  22100045   

        codeDesc abnormal_cd  
0  Levothyroxine        None  
1  Levothyroxine        None  
2   Levofloxacin        None  
3  Levothyroxine        None  
4     Prednisone        None

In [18]:
#merge with lab dict data
result_df_lab=result_df[result_df['type']=='lab']
merged_df_lab=idx2code(result_df_lab,dict_lab,cols=['idx','cd_value','LOINC_CMPNT_DESC'],idx_offset=71916)
merged_df_lab.head()

/home/jovyan/tmp/ipykernel_8481/424982093.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['codeIndex'] = df['codeIndex'] + idx_offset


mcid         dos  codeIndex  age  gender type    value         code  \
0  447975377  2019-10-25      76380   74       1  lab  0.00000  29771-3_UNK   
1  320784003  2018-05-01      75240   23       0  lab -0.44459   2345-7_UNK   
2  320784003  2018-05-01      74416   23       0  lab -0.53749   1975-2_UNK   
3  320784003  2018-05-01      82883   23       0  lab  0.47931    770-8_UNK   
4  320784003  2018-05-01      82575   23       0  lab  0.46166    742-7_UNK   

                            codeDesc abnormal_cd  
0  Hemoglobin.gastrointestinal.lower        None  
1                            Glucose        None  
2                          Bilirubin        None  
3         Neutrophils/100 leukocytes        None  
4                          Monocytes        None

In [19]:
# get normal lab value using lab pair
dict_lab_pair_val=dict_lab_pair
dict_lab_pair_val.columns=dict_lab_pair.columns.str.lower()
dict_lab_pair_val = dict_lab_pair[['idx','result_mean','result_stddev']]
merged_df_lab_val = pd.merge(merged_df_lab,dict_lab_pair_val, left_on ='codeIndex' ,right_on='idx', how ='left')
cols=['mcid','dos','codeIndex','age','gender','type','value','code','codeDesc','abnormal_cd']
merged_df_lab_val['value']=merged_df_lab_val['value']*merged_df_lab_val['result_stddev']+merged_df_lab_val['result_mean']
merged_df_lab_val=merged_df_lab_val[cols] 
merged_df_lab_val['abnormal_cd'] = merged_df_lab_val['code'].str.split('_').str[1] 
merged_df_lab_val['code'] = merged_df_lab_val['code'].str.split('_').str[0]

In [22]:
merged_df=pd.concat([merged_df_diag,merged_df_proc,merged_df_gpi,merged_df_lab_val],axis=0,ignore_index=True)

In [23]:
merged_df['mcid'].nunique()

50000

In [24]:
merged_df.shape

(19762512, 10)

In [27]:
merged_df.head()

mcid         dos  codeIndex  age  gender  type  value   code  \
0  334213630  2017-10-16       2904   77       1  diag    NaN   E034   
1  334213630  2017-10-16       3648   77       1  diag    NaN  E7800   
2  334213630  2017-10-16      19659   77       1  diag    NaN  R7303   
3  334213630  2017-10-16      19863   77       1  diag    NaN  R9720   
4  334213630  2017-10-16      41665   77       1  diag    NaN    Z23   

                                   codeDesc abnormal_cd  
0             Atrophy of thyroid (acquired)        None  
1    Pure hypercholesterolemia, unspecified        None  
2                               Prediabetes        None  
3  Elevated prostate specific antigen [PSA]        None  
4                Encounter for immunization        None

In [28]:
merged_df['ID']=pd.factorize(merged_df['mcid'])[0]+1

In [29]:
merged_df.head()

mcid         dos  codeIndex  age  gender  type  value   code  \
0  334213630  2017-10-16       2904   77       1  diag    NaN   E034   
1  334213630  2017-10-16       3648   77       1  diag    NaN  E7800   
2  334213630  2017-10-16      19659   77       1  diag    NaN  R7303   
3  334213630  2017-10-16      19863   77       1  diag    NaN  R9720   
4  334213630  2017-10-16      41665   77       1  diag    NaN    Z23   

                                   codeDesc abnormal_cd  ID  
0             Atrophy of thyroid (acquired)        None   1  
1    Pure hypercholesterolemia, unspecified        None   1  
2                               Prediabetes        None   1  
3  Elevated prostate specific antigen [PSA]        None   1  
4                Encounter for immunization        None   1

In [30]:
merged_df['ID'].nunique()

50000

## saving the table into snowflake.

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import ast
from datetime import datetime

import sys
print(sys.version)

3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 08:57:19) [GCC 11.3.0]


In [3]:
# Establish snowflake connection
conn = snowflake.connector.connect(
    account = "carelon-edaprod1.privatelink",
    user = "AN587958AD",
    password = "^qf5m3nD47h",
    warehouse = "DL_AIFS_STAR_USER_WH_L",
    database = "NON_CRTFD_AIFS",
    schema = "DL_TS_STAR"
)

# Create a cursor object
cursor = conn.cursor()

In [33]:
#Write to nogbd
table_nogbd='NON_CRTFD_AIFS.DL_TS_STAR.IHAN_LLM_INPUT_DATA_TBL'
query = """
DROP TABLE IF EXISTS {0}
""".format(table_nogbd)
print(query)
cursor.execute(query)


DROP TABLE IF EXISTS NON_CRTFD_AIFS.DL_TS_STAR.IHAN_LLM_INPUT_DATA_TBL



In [36]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19762512 entries, 0 to 19762511
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   mcid         int64  
 1   dos          object 
 2   codeIndex    int64  
 3   age          int64  
 4   gender       int64  
 5   type         object 
 6   value        float64
 7   code         object 
 8   codeDesc     object 
 9   abnormal_cd  object 
 10  ID           int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 1.6+ GB


In [37]:
# Convert to string and strip whitespace
merged_df['mcid'] = merged_df['mcid'].astype(str).str.strip()
merged_df['codeIndex'] = merged_df['codeIndex'].astype(str).str.strip()
merged_df['code'] = merged_df['code'].astype(str).str.strip()
merged_df['ID'] = merged_df['ID'].astype(str).str.strip()

In [38]:
merged_df['gender'].replace({1:"Male",0:"Female"},inplace=True)
merged_df.head()

mcid         dos codeIndex  age gender  type  value   code  \
0  334213630  2017-10-16      2904   77   Male  diag    NaN   E034   
1  334213630  2017-10-16      3648   77   Male  diag    NaN  E7800   
2  334213630  2017-10-16     19659   77   Male  diag    NaN  R7303   
3  334213630  2017-10-16     19863   77   Male  diag    NaN  R9720   
4  334213630  2017-10-16     41665   77   Male  diag    NaN    Z23   

                                   codeDesc abnormal_cd ID  
0             Atrophy of thyroid (acquired)        None  1  
1    Pure hypercholesterolemia, unspecified        None  1  
2                               Prediabetes        None  1  
3  Elevated prostate specific antigen [PSA]        None  1  
4                Encounter for immunization        None  1

In [41]:
#Write to nogbd
query = """
CREATE TABLE IF NOT EXISTS {0} (mcid STRING,dos STRING,codeIndex STRING,age NUMBER,gender STRING,type STRING,value FLOAT,code STRING,codeDesc STRING,abnormal_cd STRING,ID NUMBER)
""".format(table_nogbd)
cursor.execute(query)
success, nchunks, nrows, _ = write_pandas(conn, merged_df, table_nogbd, quote_identifiers=False)

In [8]:
df=pd.read_sql_query("""select * from NON_CRTFD_AIFS.DL_TS_STAR.IHAN_LLM_INPUT_DATA_TBL limit 100000""",conn)

/home/jovyan/tmp/ipykernel_82729/2332346494.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query("""select * from NON_CRTFD_AIFS.DL_TS_STAR.IHAN_LLM_INPUT_DATA_TBL limit 100000""",conn)


In [9]:
df.head()

MCID         DOS CODEINDEX  AGE GENDER  TYPE VALUE   CODE  \
0  334213630  2017-10-16      2904   77   Male  diag  None   E034   
1  334213630  2017-10-16      3648   77   Male  diag  None  E7800   
2  334213630  2017-10-16     19659   77   Male  diag  None  R7303   
3  334213630  2017-10-16     19863   77   Male  diag  None  R9720   
4  334213630  2017-10-16     41665   77   Male  diag  None    Z23   

                                   CODEDESC ABNORMAL_CD  ID  
0             Atrophy of thyroid (acquired)        None   1  
1    Pure hypercholesterolemia, unspecified        None   1  
2                               Prediabetes        None   1  
3  Elevated prostate specific antigen [PSA]        None   1  
4                Encounter for immunization        None   1

In [10]:
df=df.sample(frac=1)

In [11]:
df['MCID'].nunique()

727

In [14]:
import pandas as pd
import numpy as np

def sample_member_records(df, member_col='member_id', n_members=100, random_state=None):
    """
    Select all records for a random sample of members from a DataFrame.
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing all records for the sampled members
    """
    # Get unique member IDs
    unique_members = df[member_col].unique()
    
    # Ensure we don't try to sample more members than exist
    n_members = min(n_members, len(unique_members))
    
    rng = np.random.RandomState(random_state)
    
    # Randomly sample member IDs
    sampled_members = rng.choice(
        unique_members, 
        size=n_members, 
        replace=False
    )
    
    # Filter DataFrame to only include records for sampled members
    sampled_df = df[df[member_col].isin(sampled_members)]
    sampled_df.columns =[i.lower() for i in sampled_df.columns]
    
    return sampled_df

In [15]:
sampled_data = sample_member_records(
    df=df,
    member_col='MCID',  
    n_members=100,
    random_state=42  # for reproducibility
)

In [17]:
sampled_data['mcid'].nunique()

100

In [18]:
sampled_data.head()

mcid         dos codeindex  age  gender  type value    code  \
57917   72931851  2017-07-27      4801   75  Female  diag  None    G479   
1612   221480446  2019-08-09     39360   56  Female  diag  None   T8744   
57749  399504040  2017-10-04     13420   82  Female  diag  None  M25562   
94462  377202246  2018-03-23     16808   51    Male  diag  None    N186   
24223  406020137  2017-10-02      4246   79  Female  diag  None    F329   

                                                codedesc abnormal_cd   id  
57917                        Sleep disorder, unspecified        None  403  
1612   Infection of amputation stump, left lower extr...        None   10  
57749                                  Pain in left knee        None  402  
94462                            End stage renal disease        None  694  
24223  Major depressive disorder, single episode, uns...        None  164

In [19]:
sampled_data.to_csv('LLM_data_O.csv',index=False)

In [20]:
sampled_data.columns

Index(['mcid', 'dos', 'codeindex', 'age', 'gender', 'type', 'value', 'code',
       'codedesc', 'abnormal_cd', 'id'],
      dtype='object')

In [27]:
data=sampled_data[['id','dos', 'codeindex', 'age', 'gender', 'type', 'value', 'code','codedesc', 'abnormal_cd']].reset_index(drop=True)

In [28]:
data.head()

id         dos codeindex  age  gender  type value    code  \
0  403  2017-07-27      4801   75  Female  diag  None    G479   
1   10  2019-08-09     39360   56  Female  diag  None   T8744   
2  402  2017-10-04     13420   82  Female  diag  None  M25562   
3  694  2018-03-23     16808   51    Male  diag  None    N186   
4  164  2017-10-02      4246   79  Female  diag  None    F329   

                                            codedesc abnormal_cd  
0                        Sleep disorder, unspecified        None  
1  Infection of amputation stump, left lower extr...        None  
2                                  Pain in left knee        None  
3                            End stage renal disease        None  
4  Major depressive disorder, single episode, uns...        None

In [29]:
data.to_csv('LLM_data.csv',index=False)